# PaddleSeg练习——Matting抠像与背景替换

**Matting**是一种通过计算图像的颜色和透明度从从背景中提取前景对象的技术。 这项技术被广泛用于电影等行业，实现精细地背景替换、图像构图，制造各种惊艳的视觉效果。 

Matting可以看作是一个特殊的分割问题，它旨在估计**透明度**，区分`前景`和`背景`，主要目的是拿到前景，是一项真正实用的替换背景技术。分割和Matting的一个重要区别是：分割返回的是像素分类标签，其结果是整型数据；而Matting返回的是属于前景或背景的概率`P`，从而在前景与背景交互区域产生渐变的效果，使得抠图更加自然。Matting分割模型训练完成后，对于原始图像每个位置上的像素，都将生成一个表示其`前景透明度`的值，称为 **Alpha**。 图像中所有 Alpha 值的集合称为 Alpha Matte。最后，利用Alpha将原始图像的对应位置上的像素点做pixel-wise的运算，便可实现精细的背景替换。

本文将跑通Matting并实现个人照片的抠像与背景替换。

## 一、环境配置


* 环境要求

  * PaddlePaddle >= 2.0.2

  * Python >= 3.7+
  
  * PaddleSeg
  
* 数据集

  * MODNet 开源数据集 [PPM-100](https://github.com/ZHKKKe/PPM)


In [ ]:
# 下载 paddleseg
%cd ~/work/
!git clone https://github.com/PaddlePaddle/PaddleSeg.git

In [ ]:
# 安装 paddleseg
%cd ~/work/PaddleSeg
!pip install -e .
!pip install scikit-image

In [ ]:
# 下载 PPM 数据集。你也可以使用自定义数据集。
%cd ~/work/PaddleSeg/contrib/Matting
!mkdir data 
%cd data
!wget https://paddleseg.bj.bcebos.com/matting/datasets/PPM-100.zip
!unzip PPM-100.zip
%cd ..

## 二、模型性能

[DIM-VGG16](https://paddleseg.bj.bcebos.com/matting/models/dim-vgg16.pdparams)

MODNet，在[PPM-100数据集](https://github.com/ZHKKKe/PPM)上的性能

PaddleSeg提供了三种训练好的模型，可以直接下载体验。

| 骨干网络 | SAD | MSE | 模型参数下载链接 |
| --- | --- | --- | ---|
| MobileNetV2 | 112.73 | 0.0098 | [modnet-mobilenetv2](https://paddleseg.bj.bcebos.com/matting/models/modnet-mobilenetv2.pdparams) |
| ResNet50_vd | 104.14 | 0.0090 | [modnet-resnet50_vd](https://paddleseg.bj.bcebos.com/matting/models/modnet-resnet50_vd.pdparams) |
| HRNet_W18 | 77.96 | 0.0054 | [modnet-hrnet_w18](https://paddleseg.bj.bcebos.com/matting/models/modnet-hrnet_w18.pdparams) |

## 三、训练


note: 使用--do_eval会影响训练速度及增加显存消耗，根据需求进行开闭。

``--num_workers``: 多进程数据读取，加快数据预处理速度

更多参数信息请运行如下命令进行查看:
```
python train.py --help
```

如需使用多卡训练，请用 `python -m paddle.distributed.launch` 进行启动。使用方法可参考[文档](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.3/docs/train/train.md)

In [ ]:
%cd ~/work/PaddleSeg/contrib/Matting

/home/aistudio/work/PaddleSeg/contrib/Matting


In [ ]:
!wget https://paddleseg.bj.bcebos.com/matting/models/modnet-mobilenetv2.pdparams

--2021-12-01 11:51:30--  https://paddleseg.bj.bcebos.com/matting/models/modnet-mobilenetv2.pdparams
Resolving paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleseg.bj.bcebos.com (paddleseg.bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39162180 (37M) [application/octet-stream]
Saving to: ‘modnet-mobilenetv2.pdparams’

modnet-mobilenetv2. 100%[===================>]  37.35M  16.3MB/s    in 2.3s    

2021-12-01 11:51:33 (16.3 MB/s) - ‘modnet-mobilenetv2.pdparams’ saved [39162180/39162180]



In [8]:
%cd ~/work/PaddleSeg/contrib/Matting
!python train.py \
       --config configs/modnet/modnet_mobilenetv2.yml \
       --iters 50 \
       --batch_size 1 \
       --save_interval 10 \
       --do_eval \
       --log_iters 1 \
       --num_workers 0 \
       --save_dir output

/home/aistudio/work/PaddleSeg/contrib/Matting
2021-12-01 13:12:12 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-16GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.2.0
OpenCV: 4.1.1
------------------------------------------------
2021-12-01 13:12:12 [INFO]	
---------------Config Information---------------
batch_size: 1
iters: 50
lr_scheduler:
  boundaries:
  - 40000
  - 80000
  type: PiecewiseDecay
  values:
  - 0.02
  - 0.002
  - 0.0002
model:
  backbone:
    pretrained: https://paddleseg.bj.bcebos.com/matting/models/MobileNetV2_pretrained/model.pdparams
    type: MobileNetV2
  pretrained: null
  type: MODNet
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05


## 四、评估


`-save_result`开启会保留图片的预测结果，可选择关闭以加快评估速度。

可以直接下载PaddleSeg提供的模型进行评估。

更多参数信息请运行如下命令进行查看:

```
python val.py --help
```

In [10]:
!python val.py \
       --config configs/modnet/modnet_mobilenetv2.yml \
       --model_path modnet-mobilenetv2.pdparams \
       --save_dir ./output/results \
       --save_results

2021-12-01 13:14:26 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 100000
lr_scheduler:
  boundaries:
  - 40000
  - 80000
  type: PiecewiseDecay
  values:
  - 0.02
  - 0.002
  - 0.0002
model:
  backbone:
    pretrained: https://paddleseg.bj.bcebos.com/matting/models/MobileNetV2_pretrained/model.pdparams
    type: MobileNetV2
  pretrained: null
  type: MODNet
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: data/PPM-100
  mode: train
  train_file: train.txt
  transforms:
  - type: LoadImages
  - crop_size:
    - 512
    - 512
    type: RandomCrop
  - type: RandomDistort
  - type: RandomBlur
  - type: RandomHorizontalFlip
  - type: Normalize
  type: MattingDataset
val_dataset:
  dataset_root: data/PPM-100
  get_trimap: false
  mode: val
  transforms:
  - type: LoadImages
  - short_size: 512
    type: ResizeByShort
  - mult_int: 32
    type: ResizeToIntMult
  - type: Normalize
  type: MattingDataset
  val_file: v

## 五、预测及可视化结果保存

```shell
export CUDA_VISIBLE_DEVICES=0
python predict.py \
    --config configs/modnet/modnet_mobilenetv2.yml \
    --model_path output/best_model/model.pdparams \
    --image_path data/PPM-100/val/fg/ \
    --save_dir ./output/results
```

![](https://ai-studio-static-online.cdn.bcebos.com/1210201b439a49309953eea049828f4e4dff9c59491340f6ae673a6eaefc6cb1)

如模型需要trimap信息，需要通过 `--trimap_path` 传入trimap路径。

可以直接下载PaddleSeg提供的模型进行预测。

更多参数信息请运行如下命令进行查看:

```
python predict.py --help
```

In [17]:
!python predict.py \
    --config configs/modnet/modnet_mobilenetv2.yml \
    --model_path modnet-mobilenetv2.pdparams \
    --image_path predict_data/ \
    --save_dir ./output/results

2021-12-01 13:46:55 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 100000
lr_scheduler:
  boundaries:
  - 40000
  - 80000
  type: PiecewiseDecay
  values:
  - 0.02
  - 0.002
  - 0.0002
model:
  backbone:
    pretrained: https://paddleseg.bj.bcebos.com/matting/models/MobileNetV2_pretrained/model.pdparams
    type: MobileNetV2
  pretrained: null
  type: MODNet
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: data/PPM-100
  mode: train
  train_file: train.txt
  transforms:
  - type: LoadImages
  - crop_size:
    - 512
    - 512
    type: RandomCrop
  - type: RandomDistort
  - type: RandomBlur
  - type: RandomHorizontalFlip
  - type: Normalize
  type: MattingDataset
val_dataset:
  dataset_root: data/PPM-100
  get_trimap: false
  mode: val
  transforms:
  - type: LoadImages
  - short_size: 512
    type: ResizeByShort
  - mult_int: 32
    type: ResizeToIntMult
  - type: Normalize
  type: MattingDataset
  val_file: v

#### 展示预测效果
* 单人预测效果：还不错
<div align='left'>
  <img src='./Matting/predict_data/WechatIMG10.jpg' width='300'/>
  <img src='./Matting/output/results/WechatIMG10.jpg' width='300'/>
</div>


* 多人预测效果：差强人意
<div align='left'>
  <img src='./Matting/predict_data/WechatIMG11.jpg' width='300'/>
  <img src='./Matting/output/results/WechatIMG11.jpg' width='300'/>
</div>
<div align='left'>
  <img src='./Matting/predict_data/WechatIMG12.jpg' width='300'/>
  <img src='./Matting/output/results/WechatIMG12.jpg' width='300'/>
</div>

* 小猫咪预测效果：比较差（和数据集有关系）
<div align='left'>
  <img src='./Matting/predict_data/WechatIMG14.jpg' width='300'/>
  <img src='./Matting/output/results/WechatIMG14.jpg' width='300'/>
</div>

## 六、背景替换

在得到一个性能较好的Matting模型后，就可以用它实现一些有趣的应用了。比如背景替换~

In [20]:
# 执行背景替换

%cd ~/work/PaddleSeg/contrib/Matting/ 

!python bg_replace.py \
    --config configs/modnet/modnet_mobilenetv2.yml \
    --model_path modnet-mobilenetv2.pdparams \
    --image_path predict_data/WechatIMG10.jpg \
    --bg_path predict_data/background.jpg \
    --save_dir ./output/bg_results

# 可在 ./output/bg_results 路径下查看背景替换效果

/home/aistudio/work/PaddleSeg/contrib/Matting
2021-12-01 14:05:37 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 100000
lr_scheduler:
  boundaries:
  - 40000
  - 80000
  type: PiecewiseDecay
  values:
  - 0.02
  - 0.002
  - 0.0002
model:
  backbone:
    pretrained: https://paddleseg.bj.bcebos.com/matting/models/MobileNetV2_pretrained/model.pdparams
    type: MobileNetV2
  pretrained: null
  type: MODNet
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: data/PPM-100
  mode: train
  train_file: train.txt
  transforms:
  - type: LoadImages
  - crop_size:
    - 512
    - 512
    type: RandomCrop
  - type: RandomDistort
  - type: RandomBlur
  - type: RandomHorizontalFlip
  - type: Normalize
  type: MattingDataset
val_dataset:
  dataset_root: data/PPM-100
  get_trimap: false
  mode: val
  transforms:
  - type: LoadImages
  - short_size: 512
    type: ResizeByShort
  - mult_int: 32
    type: ResizeToIntMult
  - type: 

 ## <left> **背景替换效果展示(左侧为原图，右侧为背景替换效果)**
 
 
<div align='left'>
  <img src='./Matting/predict_data/WechatIMG10.jpg' width='300'/>
  <img src='./Matting/output/bg_results/WechatIMG10.jpg' width='300'/>
</div>
  
